In [10]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score

from sklearn.linear_model import LogisticRegression

complete_data = pd.read_csv("StudentPerformanceFactors.csv")

In [5]:
!pip install --upgrade numpy
!pip install --upgrade pandas
!pip install --upgrade scikit-learn
!pip install --upgrade matplotlib


  Using cached scikit_learn-1.5.2-cp311-cp311-macosx_10_9_x86_64.whl.metadata (13 kB)
  Using cached scipy-1.14.1-cp311-cp311-macosx_10_13_x86_64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.5.2-cp311-cp311-macosx_10_9_x86_64.whl (12.1 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.14.1-cp311-cp311-macosx_10_13_x86_64.whl (39.1 MB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 22.8 MB/s eta 0:00:0031m23.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 22.7 MB/s eta 0:00:00


In [15]:
# First, take code from previous check-in to clean the data to have our get our updated data frame
# Although during this regression modeling, we will only use a subset of these rows, they are all included to create our training, validation, and test data sets\
# These are the features we selected at the beginning of the project to use in our modeling

# 1) "Learning_Disabilities" is a binary feature, so we'll use this as our response variable
# 2) I'll be using "Exam_Score" as our predictor for this classification model
column_name = "Tutoring_Sessions"
main_features = ['Learning_Disabilities', column_name]
data = complete_data[main_features]

# Using the same code from the previous check-in, we'll split the data into training, validation, and test sets
# Next, divide the new dataframe into 3 different data sets using a 60:20:20 split
# We chose 60:20:20 as opposed to 80:10:10 or somwhere in between to decrease the liklihood of overfitting, since the metrics used are potentially subseptible to overfitting


train_and_validation_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

train_df, validation_df = train_test_split(train_and_validation_df, test_size=0.25, random_state=42)

train_df.to_csv('Student_Performance_train.csv', index=False)
validation_df.to_csv('Student_Performance_validation.csv', index=False)
test_df.to_csv('Student_Performance_test.csv', index=False)

x_train = train_df[[column_name]]
y_train = train_df['Learning_Disabilities']

model = LogisticRegression()
lr_model = model.fit(x_train, y_train)

y_pred = lr_model.predict(x_train)

c_m = confusion_matrix(y_train, y_pred)
print(c_m)


[[3526    0]
 [ 437    0]]
